# E2B Code Sandbox with Google Gemini
Execute AI-generated Python code in a secure sandbox environment.

## Setup and Imports

In [1]:
"""E2B Code Sandbox with Google Gemini."""
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import base64
import pandas as pd

root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root_dir)

from dotenv import load_dotenv
load_dotenv()

from e2b_code_interpreter import Sandbox
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage

## Initialize Model and Sandbox

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

sbx = Sandbox.create()
print("Sandbox created")

Sandbox created


## Helper Functions

In [3]:
def get_dataset_info(file_path):
    """Get basic dataset info."""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, nrows=3)
    else:
        df = pd.read_excel(file_path, nrows=3)

    return f"Columns: {list(df.columns)}\nSample data:\n{df.to_string()}"

## Define Tool

In [ ]:
@tool
def run_python_code(code: str):
    """Execute Python code in E2B sandbox and save chart outputs if available.
    
    IMPORTANT: The code parameter must be valid, executable Python code only.
    - Do NOT include markdown formatting (no ```python or ```)
    - Do NOT include explanations or comments outside the code
    - Include all necessary imports (pandas, matplotlib, numpy, etc.)
    - Use the exact dataset path provided in the system prompt
    - For visualizations, end with: display(plt.gcf())

    Args:
        code: Valid executable Python code as a plain string

    Returns:
        Execution result or error message
        
    Example:
        code = "import pandas as pd\\nimport matplotlib.pyplot as plt\\ndf = pd.read_csv('/path/to/data.csv')\\nplt.plot(df['x'], df['y'])\\ndisplay(plt.gcf())"
    """
    print('Running code in sandbox....')
    execution = sbx.run_code(code)

    print('Code execution finished!')

    if execution.error:
        return f"Error: {execution.error.name}\nValue: {execution.error.value}"

    results = []
    for idx, result in enumerate(execution.results):
        if result.png:
            filename = f'chart-{idx}.png'
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(result.png))
            results.append(f'Chart saved to {filename}')

    return "\n".join(results) if results else "Code executed successfully"

## Upload Dataset and Run Analysis

In [5]:
data_file = "./data/IMDB-Movie-Data.csv"

filename = os.path.basename(data_file)
with open(data_file, "rb") as f:
    sandbox_file = sbx.files.write(f"data/{filename}", f)

print(f"Uploaded: {sandbox_file.path}")

Uploaded: /home/user/data/IMDB-Movie-Data.csv


## Create Agent and Execute Query

In [6]:
dataset_info = get_dataset_info(data_file)

system_prompt = f"""You are a Python code generator for data analysis.

Dataset location: {sandbox_file.path}
{dataset_info}

CRITICAL RULES:
1. Generate ONLY executable Python code - no explanations, no markdown, no comments outside code
2. Import all required libraries (pandas, matplotlib, numpy, etc.)
3. Load data from the dataset path provided
4. For plots, MUST end with: display(plt.gcf())
5. Use the run_python_code tool with ONLY the code string

Example format:
```python
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('{sandbox_file.path}')
# your analysis code
plt.figure(figsize=(10,6))
# your plotting code
display(plt.gcf())
```"""

agent = create_agent(
    model=model,
    tools=[run_python_code],
    system_prompt=system_prompt
)



In [7]:
query = "Create a line chart showing average ratings over years"
result = agent.invoke({"messages": [HumanMessage(content=query)]})

response = result['messages'][-1].text
print(f"\nResponse:\n{response}")

Running code in sandbox....
Code execution finished!

Response:



In [8]:
result

{'messages': [HumanMessage(content='Create a line chart showing average ratings over years', additional_kwargs={}, response_metadata={}, id='7b9615b6-8ad9-45e7-9bb7-fffaaec05fe2'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'run_python_code', 'arguments': '{"code": "\\nimport pandas as pd\\nimport matplotlib.pyplot as plt\\n\\ndf = pd.read_csv(\'/home/user/data/IMDB-Movie-Data.csv\')\\n\\naverage_ratings_over_year = df.groupby(\'Year\')[\'Rating\'].mean()\\n\\nplt.figure(figsize=(12, 6))\\nplt.plot(average_ratings_over_year.index, average_ratings_over_year.values, marker=\'o\')\\nplt.title(\'Average Movie Ratings Over Years\')\\nplt.xlabel(\'Year\')\\nplt.ylabel(\'Average Rating\')\\nplt.grid(True)\\ndisplay(plt.gcf())\\n"}'}, '__gemini_function_call_thought_signatures__': {'cb8c569b-8021-4a29-a4cf-197e6e334177': 'CqgCAXLI2nxQbRHdL5JrJ4ikoRbkibpgnIXgufpuGw0k4qIc2FLK3WKzvw/KxpTNYDeSmdviRHyaFuwCxleXMvztqGWxrDBRtbD9DrTvBkNalf8k3n1JGDJCKaJl6muvSD+cnH2sMfdWPUum/Vom